In [1]:
# Need some packages to talk to Kafka.
# if we have no jars file it will automatic download
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2 pyspark-shell'

# or set straight with spark configuration
# spark = SparkSession.builder.appName('Streaming')\
#     .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
#     .getOrCreate()

In [2]:
from pyspark.sql import SparkSession
# create spark
spark = SparkSession.builder.appName('Streaming')\
    .config('spark.jars.packages', 'org.apache.spark:spark-streaming-kafka-0-10_2.12:3.1.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')\
    .getOrCreate()

22/03/07 16:48:28 WARN Utils: Your hostname, HuyLes-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.2 instead (on interface en0)
22/03/07 16:48:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/huyle/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/huyle/.ivy2/cache
The jars for the packages stored in: /Users/huyle/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1afedc69-b582-4e2c-9205-bd0154049665;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 1198ms :: artifa

In [3]:
# read stream from kafka
df = spark \
      .readStream \
      .format("kafka") \
      .option("kafka.bootstrap.servers", "localhost:9092") \
      .option("subscribe", "DemoTopic") \
      .option("startingOffsets", "earliest") \
      .load()

# .option("failOnDataLoss", "false")\
# df_uber = df.select(col("key").cast("string"),from_json(col("value").cast("string"), schema).alias("value")).select("value.*")
# df_uber.printSchema()
# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") # convert key-value from binary to string type
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
from pyspark.sql.functions import col, to_timestamp

# Parsing the messeage value into dataframe
df_uber = df.select(col("value").cast("string")).alias("csv").select("csv.*")
df_uber2 = df_uber.selectExpr("split(value,',')[0] as dt",
                               "split(value,',')[1] as lat",
                               "split(value,',')[2] as lon",
                               "split(value,',')[3] as base")
df_uber2.printSchema()

root
 |-- dt: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- base: string (nullable = true)



In [5]:
# Show df
# df_uber2.writeStream.format("console").outputMode("append").start()

In [6]:
# write stream dataframe to hdfs
df_uber2.writeStream.format("csv") \
                    .option("path", "hdfs://localhost:9000/raw/") \
                    .option("checkpointLocation", "hdfs://localhost:9000/checkpoints/") \
                    .outputMode("append") \
                    .start()

In [7]:
# spark.stop()

22/03/07 17:32:34 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-c1b242c0-9315-48ff-9d4d-afeccb02f225--1847986763-driver-0-1, groupId=spark-kafka-source-c1b242c0-9315-48ff-9d4d-afeccb02f225--1847986763-driver-0] Connection to node 1 (/127.0.0.1:9092) could not be established. Broker may not be available.
22/03/07 17:32:35 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-c1b242c0-9315-48ff-9d4d-afeccb02f225--1847986763-driver-0-1, groupId=spark-kafka-source-c1b242c0-9315-48ff-9d4d-afeccb02f225--1847986763-driver-0] Connection to node 1 (/127.0.0.1:9092) could not be established. Broker may not be available.
22/03/07 17:32:36 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-c1b242c0-9315-48ff-9d4d-afeccb02f225--1847986763-driver-0-1, groupId=spark-kafka-source-c1b242c0-9315-48ff-9d4d-afeccb02f225--1847986763-driver-0] Connection to node 1 (/127.0.0.1:9092) could not be established. Broker may not be available.
22/03/07 17:32:37 WARN